In [17]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Configuration d'affichage
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')
sns.set_palette("crest")

In [18]:
# --- 1. Chargement du jeu de données et nettoyage des noms ---

try:
    df = pd.read_csv("autos.csv", encoding='latin1')
except FileNotFoundError:
    print("Erreur : Le fichier 'autos.csv' est introuvable. Veuillez le placer dans le répertoire courant.")
    exit()

# Nettoyage des noms des colonnes
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

print(f"Jeu de données chargé : {df.shape[0]} lignes et {df.shape[1]} colonnes")
print("\n🔎 Informations initiales :")
df.info()

Jeu de données chargé : 371528 lignes et 21 colonnes

🔎 Informations initiales :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   index                371528 non-null  int64 
 1   datecrawled          371528 non-null  object
 2   name                 371528 non-null  object
 3   seller               371528 non-null  object
 4   offertype            371528 non-null  object
 5   price                371528 non-null  int64 
 6   abtest               371528 non-null  object
 7   vehicletype          333659 non-null  object
 8   yearofregistration   371528 non-null  int64 
 9   gearbox              351319 non-null  object
 10  powerps              371528 non-null  int64 
 11  model                351044 non-null  object
 12  kilometer            371528 non-null  int64 
 13  monthofregistration  371528 non-null  int64 
 14  fue

In [19]:
# --- 2. Suppression des colonnes non pertinentes ---

colonnes_a_supprimer = [
    'index', 
    'datecrawled', 
    'name', 
    'seller', 
    'offertype', 
    'abtest', 
    'datecreated', 
    'nrofpictures', 
    'postalcode', 
    'lastseen'
]

# Suppression des colonnes
df.drop(columns=colonnes_a_supprimer, inplace=True)

print(f"\n✅ Colonnes non pertinentes supprimées : {colonnes_a_supprimer}")
print(f"Colonnes restantes : {df.columns.tolist()}")


✅ Colonnes non pertinentes supprimées : ['index', 'datecrawled', 'name', 'seller', 'offertype', 'abtest', 'datecreated', 'nrofpictures', 'postalcode', 'lastseen']
Colonnes restantes : ['price', 'vehicletype', 'yearofregistration', 'gearbox', 'powerps', 'model', 'kilometer', 'monthofregistration', 'fueltype', 'brand', 'notrepaireddamage']


In [20]:
# --- 3. Traitement des Doublons ---

nb_doublons = df.duplicated().sum()
df = df.drop_duplicates()
print(f"\nNombre de doublons supprimés : {nb_doublons}")
print(f"Nouvelle taille du dataset : {df.shape}")


Nombre de doublons supprimés : 28503
Nouvelle taille du dataset : (343025, 11)


In [21]:
# --- 4. Traitement des Doublons ---

lignes_doublons = df.duplicated().sum()
df = df.drop_duplicates()
print(f"Nombre de doublons supprimés : {lignes_doublons}. Taille actuelle : {df.shape}")




Nombre de doublons supprimés : 0. Taille actuelle : (343025, 11)


In [22]:
# --- 5. Traitement des valeurs manquantes (Suppression + Imputation simple) ---

# a) Suppression des lignes sans information essentielle (Modèle et Boîte de Vitesse)
# On choisit de supprimer les NaNs pour les features les plus critiques.
lignes_avant_drop = df.shape[0]
df = df.dropna(subset=['model', 'gearbox'])
lignes_supprimees_essentiel = lignes_avant_drop - df.shape[0]

print(f"\nLignes sans 'model' ou 'gearbox' supprimées : {lignes_supprimees_essentiel}")


Lignes sans 'model' ou 'gearbox' supprimées : 33604


In [23]:
# --- 6. Traitement des valeurs aberrantes (Outliers) ---

lignes_avant_outlier = df.shape[0]

# a) Filtrage du prix (variable cible)
# On supprime les prix <= 100 (non significatifs, erreurs ou prix symboliques)
df = df[df['price'] > 100]

# b) Filtrage de l'année d'enregistrement
# On garde les années réalistes (entre 1950 et 2025)
df = df[(df['yearofregistration'] >= 1950) & (df['yearofregistration'] <= 2025)]

# c) Filtrage de la puissance moteur (powerps)
# On supprime les valeurs non réalistes ou extrêmes (Puissance entre 50 PS et 500 PS)
df = df[(df['powerps'] >= 50) & (df['powerps'] <= 500)].copy()

lignes_supprimees_outlier = lignes_avant_outlier - df.shape[0]

print(f"\nLignes avec valeurs aberrantes (prix, année, puissance) supprimées : {lignes_supprimees_outlier}")


Lignes avec valeurs aberrantes (prix, année, puissance) supprimées : 34576


In [24]:
print("\n--- Résumé du nettoyage de données ---")
print(f"Dimensions finales du dataset : {df.shape}")
print(f"Colonnes finales : {df.columns.tolist()}")
print("\nStatistiques des variables numériques restantes (pour vérification):")
print(df.describe().T)

print("\nLe jeu de données est maintenant nettoyé et prêt pour l'analyse.")


--- Résumé du nettoyage de données ---
Dimensions finales du dataset : (274845, 11)
Colonnes finales : ['price', 'vehicletype', 'yearofregistration', 'gearbox', 'powerps', 'model', 'kilometer', 'monthofregistration', 'fueltype', 'brand', 'notrepaireddamage']

Statistiques des variables numériques restantes (pour vérification):
                        count           mean            std     min       25%  \
price                274845.0   10043.151365  550302.173149   101.0    1500.0   
yearofregistration   274845.0    2003.817828       6.791636  1950.0    1999.0   
powerps              274845.0     127.372377      58.959170    50.0      84.0   
kilometer            274845.0  125518.565009   39127.902222  5000.0  100000.0   
monthofregistration  274845.0       6.033186       3.555603     0.0       3.0   

                          50%       75%         max  
price                  3550.0    8000.0  99999999.0  
yearofregistration     2004.0    2008.0      2019.0  
powerps              